## Желтое Такси Нью-Йорка - Прогнозирование рядов
### Представление проекта: интерактивный прогноз для выбранной на карте области

In [1]:
%matplotlib inline
from matplotlib.pyplot import *
from IPython.core.pylabtools import figsize

from IPython.display import HTML, clear_output, Javascript, Latex
from ipywidgets import widgets
from yellow_taxi import *

wait_message = HTML('<center>Building model</center>')

cell = None
hour = 4*24+12

zoom = 10
center = ESB

def RebuildModel():
    global m
    m = LAR(agg[cell][:tr], nar = 500).fit()

def RebuildPredictions():
    global pred
    m.reset(tr)
    if hour>0: m.update(agg[cell][tr:tr+hour])
    pred = m.predict(tr, tr+tt-1)

def SetCell(cell_ll_):
    
    global cell_ll
    cell_ll = cell_ll_
    
    global cell
    cell=regid_by_ll(cell_ll)
    RebuildModel()
    RebuildPredictions()

def SetHour(hour_):
    global hour
    hour = hour_
    RebuildPredictions()

def MapAction(zoom_, lat_, lng_):
    global zoom, center
    zoom, center = zoom_, LatLong(lat_, lng_)

def OnMapDblClick(lat, lng):
    with console:
        clear_output(True)
        display(wait_message)
        SetCell(LatLong(lat, lng))
        clear_output()
    render_plot()
    
def OnSlide(change):
    SetHour(change['new'])
    UpdateHourLabel()
    render_plot()
    render_map()

    
def render_map():
    with out_map:
        clear_output(True)
        data = agg.iloc[tr+hour]
        max_ = np.max(agg.values)
        rects = [
            [reg_by_nll(nll_by_regid(cell_id)), '#FF0000', round(0.1 + 0.8*(data[cell_id]/max_),1)] 
            for cell_id in data.index
            if data[cell_id]>0
        ]
        
        display(google_map(
            center, zoom,
            map_action='MapAction',
            size=(750,250), 
            box=NY, box_action='OnMapDblClick', box_marker=cell_ll,
            rectangles = rects
        ))
        
        print 'The current load is represented on the map.'

def render_plot():
    with out_plot:
        clear_output(True)
        figsize(12.7, 3)
        plot(agg[cell][tr:], label='Real')
        plot(pred[:hour], label='Predict on Real')
        plot(pred[hour:], label='Predict')
        
        axvline(agg.index[tr+hour], color='y')
        title('Real data and prognosis for marked area (cell %d) for June 2016.\nTo select area double click inside red box on map below.' % cell)
        legend(loc=1)
        show()

def UpdateHourLabel():
    global hour_label
    hour_label.value = str(agg.index[tr+hour])
        
def render_widgets():
    with out_widgets:
        clear_output(True)
        
        global hour_label
        hour_label = widgets.Label()
        UpdateHourLabel()
        
        slider = widgets.IntSlider(
                hour, 0, tt-1, 
                description = '', 
                readout=False,
                continuous_update=False,
                layout = widgets.Layout(width='450px')
        )
        
        slider.observe(OnSlide, names='value')
        
        display(widgets.HBox([
            hour_label,
            slider
        ], layout = widgets.Layout(width='750px')))

console = widgets.Output()
display(console)

with console:
    display(wait_message)

    agg = load_aggregated(-1,2016,6, verbose = False)

    tr = len(agg[:'2016-06-15'])
    tt = len(agg[tr:])
    
    SetCell(ESB)
    clear_output()

out_map = widgets.Output()
out_plot = widgets.Output()
out_widgets = widgets.Output()

display(out_plot)
display(out_widgets)
display(out_map)

render_map()
render_plot()
render_widgets()

Output()

Output()

Output()

Output()

** Описание **

Делаем прогноз на июнь 2016 года при помощи модели, обученной на данных до мая 2016 (включительно). Обучения и предсказания делаются индивидуально для каждого региона.

** Как пользоваться **

Запустить код в ячейке.

Выбор интересующего региона осуществляется двойным кликом на интерактивной карте. При этом автоматически определится регион, будет построена модель для него и на графике будет отображен ряд, соответстующий количеству поездок из выбранного региона и прогноз. По причине построения модели отзыв может быть достаточно долгим. Можно было бы заранее подготовить модели для всех регионов, но такой подход потребует больше памяти.

С помощью слайдера можно выбрать дату и время. На карте будет отображен спрос на такси по всему городу в соответствии с этим моментом времени. На графике отображаются реальный спрос (синим) и результаты прогнозирования. При этом слева от вертикальной линии (выбранный момент времени) прогноз будет рассчитан с использованием только реальных данных (т.е. мы увидим, насколько хорошо авторегресиионная модель описывает имеющиеся данные), а справа будет дан прогноз в будущее. Перемещая слайдер можно наглядно наблюдать как меняется прогноз справа по мере поступления в модель новых реальных данных.

Так выглядит программа (для тех, кому не удалось запустить ноутбук)

<img src="files/screen.png">